In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2025=pd.read_csv('../../scrapers/stats_scraper/2025_batting_stats.csv')
batter_team_2025=pd.read_csv('batter_team_2025.csv')

In [3]:
stats_2025

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_AB,home_R,home_H,home_RBI,home_HR,home_BB,home_K,home_AVG,home_OBP,home_SLG
0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,0.361,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.257,0.357,0.544
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,0.387,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.409,0.495,0.766
2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,0.413,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.238,0.304,0.397
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,0.287,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.345,0.393,0.500
4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,0.308,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.240,0.336,0.426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7550,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,0.350,...,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.216,0.263,0.443
7551,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,0.328,...,4.0,0.0,3.0,0.0,0.0,0.0,1.0,0.172,0.303,0.287
7552,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,NaN,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,0.274,0.333
7553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,1.0,1.0,0.0,0.0,0.0,2.0,0.219,0.219,0.281


In [4]:
# stats_2025 = stats_2025.drop(columns=['Team_x', 'Team_y','away_player','home_player'])


In [5]:
#normalizing the names, getting rid of accents
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [6]:
batter_team_2025['Player'] = batter_team_2025['Player'].apply(normalize_name)

In [7]:
#getting rid of the # and the * from the names
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

In [8]:
batter_team_2025['Player'] = batter_team_2025['Player'].apply(clean_name_symbols)

In [9]:
# multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
# filtered_df_2 = batter_team_2025[~batter_team_2025['Team'].isin(multi_team_flags)].copy()
# batter_team_2025 = filtered_df_2.drop_duplicates(subset='Player', keep='last').reset_index(drop=True)

In [10]:
batter_team_2025

,Unnamed: 0,Player,Team
0,0,Jarren Duran,BOS
1,1,Rafael Devers,BOS
2,2,Corbin Carroll,ARI
3,3,Kyle Tucker,CHC
4,4,Alex Bregman,BOS
...,...,...,...
533,533,Nick Sandlin,TOR
534,534,Tanner Scott,LAD
535,535,Ryan Thompson,ARI
536,536,Ryan Walker,SFG


In [11]:
# Step 1: Safely extract short names like "P. Sandoval"
# def clean_name(raw):
#     if not isinstance(raw, str):
#         return None
#     match = re.match(r"([A-Za-z]{1,2}\.\s[A-Za-z'-]+)", raw)
#     return match.group(1) if match else None

def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # 1) initial-style (one or more letter-dot sequences, space, surname+optional code)
    m = re.search(r"((?:[A-Za-z]\.)+\s[A-Za-z'-]+)", raw)
    if m:
        name = m.group(1)
        # strip any trailing digits/uppercase letters/hyphens
        strip = re.match(r"^((?:[A-Za-z]\.)+\s[A-Za-z'-]+?)(?:[0-9A-Z-]+)$", name)
        return strip.group(1) if strip else name

    # 2) plain First Last but only up to just before any digit or hyphen
    m2 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)(?=[0-9-])", raw)
    if m2:
        return m2.group(1)

    # 3) fallback to a simple First Last
    m3 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)", raw)
    if m3:
        return m3.group(1)

    return None

stats_2025['short_name'] = stats_2025['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['Player'].tolist()
stats_2025['matched_name'] = stats_2025['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df = stats_2025.merge(batter_team_2025, left_on='matched_name', right_on='Player', how='left')

In [12]:
merged_df

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_BB,home_K,home_AVG,home_OBP,home_SLG,short_name,matched_name,Unnamed: 0,Player,Team
0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,0.361,...,0.0,0.0,0.257,0.357,0.544,F. Lindor,Francisco Lindor,5.0,Francisco Lindor,NYM
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,0.387,...,0.0,1.0,0.409,0.495,0.766,J. Soto,Juan Soto,7.0,Juan Soto,NYM
2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,0.413,...,0.0,1.0,0.238,0.304,0.397,P. Alonso,Pete Alonso,9.0,Pete Alonso,NYM
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,0.287,...,0.0,1.0,0.345,0.393,0.500,B. Nimmo,Brandon Nimmo,61.0,Brandon Nimmo,NYM
4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,0.308,...,0.0,0.0,0.240,0.336,0.426,M. Vientos,Mark Vientos,101.0,Mark Vientos,NYM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7583,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,0.350,...,0.0,0.0,0.216,0.263,0.443,L. Urias,Luis Urias,224.0,Luis Urias,ATH
7584,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,0.328,...,0.0,1.0,0.172,0.303,0.287,N. Kurtz,Nick Kurtz,325.0,Nick Kurtz,ATH
7585,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,NaN,...,0.0,0.0,0.250,0.274,0.333,None,None,NaN,NaN,NaN
7586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,0.219,0.219,0.281,None,None,NaN,NaN,NaN


In [13]:
#dropping the collumns we used to match to then match the other column
merged_df = merged_df.drop(columns=['short_name', 'matched_name','Unnamed: 0'])


In [14]:
merged_df.rename(columns={'Player': 'away_player', 'Team':'away_team'}, inplace=True)

In [15]:
merged_df

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_H,home_RBI,home_HR,home_BB,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team
0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,0.361,...,1.0,0.0,0.0,0.0,0.0,0.257,0.357,0.544,Francisco Lindor,NYM
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,0.387,...,0.0,0.0,0.0,0.0,1.0,0.409,0.495,0.766,Juan Soto,NYM
2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,0.413,...,0.0,0.0,0.0,0.0,1.0,0.238,0.304,0.397,Pete Alonso,NYM
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,0.287,...,0.0,0.0,0.0,0.0,1.0,0.345,0.393,0.500,Brandon Nimmo,NYM
4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,0.308,...,1.0,0.0,0.0,0.0,0.0,0.240,0.336,0.426,Mark Vientos,NYM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7583,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,0.350,...,1.0,0.0,0.0,0.0,0.0,0.216,0.263,0.443,Luis Urias,ATH
7584,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,0.328,...,3.0,0.0,0.0,0.0,1.0,0.172,0.303,0.287,Nick Kurtz,ATH
7585,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.250,0.274,0.333,NaN,NaN
7586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,2.0,0.219,0.219,0.281,NaN,NaN


In [16]:
# Step 1: Safely extract short names like "P. Sandoval"
# def clean_name(raw):
#     if not isinstance(raw, str):
#         return None
#     match = re.match(r"([A-Za-z]{1,2}\.\s[A-Za-z'-]+)", raw)
#     return match.group(1) if match else None
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # 1) initial-style (one or more letter-dot sequences, space, surname+optional code)
    m = re.search(r"((?:[A-Za-z]\.)+\s[A-Za-z'-]+)", raw)
    if m:
        name = m.group(1)
        # strip any trailing digits/uppercase letters/hyphens
        strip = re.match(r"^((?:[A-Za-z]\.)+\s[A-Za-z'-]+?)(?:[0-9A-Z-]+)$", name)
        return strip.group(1) if strip else name

    # 2) plain First Last but only up to just before any digit or hyphen
    m2 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)(?=[0-9-])", raw)
    if m2:
        return m2.group(1)

    # 3) fallback to a simple First Last
    m3 = re.search(r"([A-Za-z]+ [A-Za-z'-]+)", raw)
    if m3:
        return m3.group(1)

    return None

merged_df['short_name'] = merged_df['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    # Try to match by last name + same first initial
    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # match first initial
            return name

    # Fallback to fuzzy match if nothing matched
    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['Player'].tolist()
merged_df['matched_name'] = merged_df['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_5 = merged_df.merge(batter_team_2025, left_on='matched_name', right_on='Player', how='left')

In [17]:
#dropping the collumns we used to match to then match the other column
merged_df_5 = merged_df_5.drop(columns=['short_name', 'matched_name', 'Unnamed: 0'])

In [18]:
merged_df_5.rename(columns={'Player': 'home_player', 'Team':'home_team'}, inplace=True)

In [19]:
import pandas as pd

def assign_game_ids_by_scan(df: pd.DataFrame,
                            away_col: str    = 'away_hitters',
                            home_col: str    = 'home_hitters',
                            summary_val: str = 'team',
                            id_col: str      = 'gameID') -> pd.DataFrame:
    """
    Walk down the DataFrame row by row.  
    - Start gameID = 1.  
    - At each row: assign current gameID.  
    - If row[away_col] == summary_val, mark away_seen.  
      If row[home_col] == summary_val, mark home_seen.  
    - As soon as both away_seen & home_seen are True, increment gameID, reset both flags.
    """
    df = df.copy()
    game_id = 1
    away_seen = False
    home_seen = False
    ids = []

    for _, row in df.iterrows():
        # assign the current game_id
        ids.append(game_id)

        # check for each summary marker
        if str(row[away_col]).strip().lower() == summary_val:
            away_seen = True
        if str(row[home_col]).strip().lower() == summary_val:
            home_seen = True

        # once both sides have had their 'team' row, we end the block
        if away_seen and home_seen:
            game_id  += 1
            away_seen = False
            home_seen = False

    df[id_col] = ids
    return df

# Apply it to your merged_df_5:
merged_df_5 = assign_game_ids_by_scan(
    merged_df_5,
    away_col='away_hitters',
    home_col='home_hitters',
    summary_val='team',
    id_col='gameID'
)

# Quick check
print(merged_df_5[['away_hitters','home_hitters','gameID']].iloc[150:180])


            away_hitters      home_hitters  gameID
150            J. BaezSS        M. Rojas2B      13
151   aZ. McKinstryPH-SS        A. PagesLF      13
152           J. RogersC              team      13
153        R. KreidlerCF               NaN      13
154   bK. CarpenterPH-LF               NaN      13
155                 team               NaN      13
156            I. HappLF        K. Marte2B      14
157          K. TuckerRF      C. CarrollRF      14
158          S. SuzukiDH  L. Gurriel Jr.LF      14
159           M. Busch1B       J. Naylor1B      14
160         D. SwansonSS        G. MorenoC      14
161         N. Hoerner2B       E. Suarez3B      14
162  P. Crow-ArmstrongCF      R. GrichukDH      14
163            M. Shaw3B    aP. SmithPH-DH      14
164            M. AmayaC     J. McCarthyCF      14
165                 team      G. PerdomoSS      14
166                  NaN              team      14
167          L. ButlerRF       V. RoblesRF      15
168          B. RookerDH    J. 

In [20]:
merged_df_5

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_BB,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team,home_player,home_team,gameID
0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,0.361,...,0.0,0.0,0.257,0.357,0.544,Francisco Lindor,NYM,Ben Rice,NYY,1
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,0.387,...,0.0,1.0,0.409,0.495,0.766,Juan Soto,NYM,Aaron Judge,NYY,1
2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,0.413,...,0.0,1.0,0.238,0.304,0.397,Pete Alonso,NYM,Cody Bellinger,NYY,1
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,0.287,...,0.0,1.0,0.345,0.393,0.500,Brandon Nimmo,NYM,Paul Goldschmidt,NYY,1
4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,0.308,...,0.0,0.0,0.240,0.336,0.426,Mark Vientos,NYM,Jasson Dominguez,NYY,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,0.350,...,0.0,0.0,0.216,0.263,0.443,Luis Urias,ATH,Ryan O'Hearn,BAL,630
7609,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,0.328,...,0.0,1.0,0.172,0.303,0.287,Nick Kurtz,ATH,Michael Conforto,LAD,630
7610,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,NaN,...,0.0,0.0,0.250,0.274,0.333,NaN,NaN,Miguel Rojas,LAD,630
7611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,0.219,0.219,0.281,NaN,NaN,Chris Taylor,LAD,630


In [22]:
for col in ['away_team','home_team']:
    merged_df_5[col] = (
        merged_df_5
        .groupby('gameID')[col]
        .transform(lambda x: x.ffill().bfill())
    )

In [23]:
merged_df_5.to_csv('2025_batting_stats_edited.csv')

In [24]:
merged_df_5

,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,away_OBP,...,home_BB,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team,home_player,home_team,gameID
0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,0.361,...,0.0,0.0,0.257,0.357,0.544,Francisco Lindor,NYM,Ben Rice,NYY,1
1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,0.387,...,0.0,1.0,0.409,0.495,0.766,Juan Soto,NYM,Aaron Judge,NYY,1
2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,0.413,...,0.0,1.0,0.238,0.304,0.397,Pete Alonso,NYM,Cody Bellinger,NYY,1
3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,0.287,...,0.0,1.0,0.345,0.393,0.500,Brandon Nimmo,NYM,Paul Goldschmidt,NYY,1
4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,0.308,...,0.0,0.0,0.240,0.336,0.426,Mark Vientos,NYM,Jasson Dominguez,NYY,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,0.350,...,0.0,0.0,0.216,0.263,0.443,Luis Urias,ATH,Ryan O'Hearn,BAL,630
7609,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,0.328,...,0.0,1.0,0.172,0.303,0.287,Nick Kurtz,ATH,Michael Conforto,LAD,630
7610,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,NaN,...,0.0,0.0,0.250,0.274,0.333,NaN,ATH,Miguel Rojas,LAD,630
7611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,0.219,0.219,0.281,NaN,ATH,Chris Taylor,LAD,630


In [25]:
# merged_df_5.to_csv('batter_stats_2025.csv')